In [1]:
# import libraries

import torch
import numpy as np
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
import numpy as np 
import math

import shutil
import os
import time
import sys
from pathlib import Path
import functools
from collections import Counter
print = functools.partial(print, flush=True)

from lib_net import *
from lib_util import *

# interest_num = [0,1,2,3,4,5,6,7,8,9]
interest_num = [3,6]
img_size = 4
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 16
inference_batch_size = 16
num_f1 = 16
num_f2 = len(interest_num)
init_lr = 0.01


save_to_file = False
if save_to_file:
    sys.stdout = open(save_path+"/log", 'w')
save_path = "./model/"+os.path.basename(sys.argv[0])+"_"+time.strftime("%Y_%m_%d-%H_%M_%S")
Path(save_path).mkdir(parents=True, exist_ok=True)

resume_path = ""
training = True
max_epoch = 10


criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()



print("="*100)
print("Training procedure for Quantum Computer:")
print("\tStart at:",time.strftime("%m/%d/%Y %H:%M:%S"))
print("\tProblems and issues, please contact Dr. Weiwen Jiang (wjiang2@nd.edu)")
print("\tEnjoy and Good Luck!")
print("="*100)
print()

Training procedure for Quantum Computer:
	Start at: 11/11/2020 18:29:54
	Problems and issues, please contact Dr. Weiwen Jiang (wjiang2@nd.edu)
	Enjoy and Good Luck!



In [2]:
# def modify_target(target):
#     for j in range(len(target)):
#         for idx in range(len(interest_num)):
#             if target[j] == interest_num[idx]:
#                 target[j] = idx
#                 break
# 
#     new_target = torch.zeros(target.shape[0],2)
# 
#     for i in range(target.shape[0]):        
#         if target[i].item() == 0:            
#             new_target[i] = torch.tensor([1,0]).clone()     
#         else:
#             new_target[i] = torch.tensor([0,1]).clone()
# 
#     return target,new_target
# 
# def select_num(dataset,interest_num):
#     labels = dataset.targets #get labels
#     labels = labels.numpy()
#     idx = {}
#     for num in interest_num:
#         idx[num] = np.where(labels == num)
#         
#     fin_idx = idx[interest_num[0]]
#     for i in range(1,len(interest_num)):           
#         
#         fin_idx = (np.concatenate((fin_idx[0],idx[interest_num[i]][0])),)
#     
#     fin_idx = fin_idx[0]    
#     
#     dataset.targets = labels[fin_idx]
#     dataset.data = dataset.data[fin_idx]
#     
#     # print(dataset.targets.shape)
#     
#     dataset.targets,_ = modify_target(dataset.targets,interest_num)
#     # print(dataset.targets.shape)
#     
#     return dataset
# 
# 
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# 
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# 
# dataset = pd.read_csv("../iris.data")
# dataset.columns = ["sepal length (cm)", 
#                    "sepal width (cm)", 
#                    "petal length (cm)", 
#                    "petal width (cm)", 
#                    "species"]
# 
# mappings = {
#    "Iris-setosa": 0,
#    "Iris-versicolor": 1,
#    "Iris-virginica": 2
# }
# dataset["species"] = dataset["species"].apply(lambda x: mappings[x])
# 
# X = dataset.drop("species",axis=1).values
# y = dataset["species"].values
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20)
# 
# X_train = torch.FloatTensor(X_train)
# y_train = torch.LongTensor(y_train)
# 
# X_test = torch.FloatTensor(X_test)
# y_test = torch.LongTensor(y_test)
# 
# train_data = torch.utils.data.TensorDataset(X_train, y_train)
# test_data = torch.utils.data.TensorDataset(X_test,y_test)
# 
# # prepare data loaders
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
#     num_workers=num_workers, shuffle=True, drop_last=True)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=inference_batch_size, 
#     num_workers=num_workers, shuffle=True, drop_last=True)



In [3]:
# convert data to torch.FloatTensor
transform = transforms.Compose([transforms.Resize((img_size,img_size)),transforms.ToTensor()])
# transform = transforms.Compose([transforms.Resize((img_size,img_size)),transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
# choose the training and test datasets
train_data = datasets.MNIST(root='../../pytorch/data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='../../pytorch/data', train=False,
                                  download=True, transform=transform)

train_data = select_num(train_data,interest_num)
test_data =  select_num(test_data,interest_num)


# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=inference_batch_size, 
    num_workers=num_workers, shuffle=True, drop_last=True)



def save_checkpoint(state, is_best, save_path, filename):
    filename = os.path.join(save_path, filename)
    torch.save(state, filename)
    if is_best:
        bestname = os.path.join(save_path, 'model_best.tar')
        shutil.copyfile(filename, bestname)



In [4]:



def train(epoch):
    model.train()
    correct = 0
    epoch_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        target,new_target = modify_target(target,interest_num)
        # 
        # data = (data-data.min())/(data.max()-data.min())
        # data = (binarize(data-0.5)+1)/2
        # 
        
        
        
        data, target = data.to(device), target.to(device)        
        optimizer.zero_grad()
        output = model(data,True)
        
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()    
        
        loss = criterion(output, target)
        epoch_loss.append(loss.item())
        loss.backward()
        
        optimizer.step()
                
        if batch_idx % 100 == 0:        
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {}/{} ({:.2f}%)'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss, correct, (batch_idx+1) * len(data),
                100. * float(correct) / float(((batch_idx+1) * len(data)) )))                
    print("-"*20,"training done, loss","-"*20)
    print("Training Set: Average loss: {}".format(round(sum(epoch_loss)/len(epoch_loss),6)))
    
accur=[]
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        target,new_target = modify_target(target,interest_num)
        
        # 
        # data = (data-data.min())/(data.max()-data.min())
        # data = (binarize(data-0.5)+1)/2
        
        data, target = data.to(device), target.to(device)
        
        # print("Debug")
        # output = model(data,2)
        # 
        # sys.exit(0)
        # data, target = Variable(data, volatile=True), Variable(target)
        output = model(data,False)
        test_loss += criterion(output, target) # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    a=100.*correct / len(test_loader.dataset)
    accur.append(a)  
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * float(correct) / float(len(test_loader.dataset))))
    
    return float(correct) / len(test_loader.dataset)




#Training


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net(img_size,[num_f1,num_f2],True,[[1, -1, 1, -1], [-1, -1]],
            True,training,False,False,False).to("cpu")


# -nn "4, 2" -bin -qt -c $dataset -s 4 -l 0.1 -ql 0.0001 -e 5 -m "2, 4"
# def __init__(self,img_size,layers,with_norm,given_ang,train_ang,training,binary,classic,debug="False"):


print("="*10,"Model Info","="*10)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)
# optimizer = torch.optim.Adam([
#                 {'params': model.fc1.parameters()},
#                 {'params': model.fc2.parameters()},
#                 {'params': model.fc3.parameters()},
#                 {'params': model.qc1.parameters(), 'lr': 1},
#                 {'params': model.qc2.parameters(), 'lr': 1},
#                 {'params': model.qc3.parameters(), 'lr': 1},
#             ], lr=0.1)

# optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

# optimizer = torch.optim.SGD([
#                 {'params': model.fc1.parameters()},
#                 {'params': model.fc2.parameters()},
#                 {'params': model.fc3.parameters()},
#                 {'params': model.qc1.parameters(), 'lr': 1},
#                 {'params': model.qc2.parameters(), 'lr': 1},
#                 {'params': model.qc3.parameters(), 'lr': 1},
#             ], lr=0.1, momentum=0.9)
# 
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, \
#         base_lr=[1e-1,1e-1,1e-1,1,1,1], \
#         max_lr=[1e-3,1e-3,1e-3,1e-2,1e-2,1e-2], \
#         step_size_up=100
#         )

milestones = [3, 5, 8]
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones, gamma=0.1)

# 
# 
# test()
# 
# 

4 [16, 2] True [[1, -1, 1, -1], [-1, -1]] True True False False False
========== Model Info ==========
Net(
  (fc0): BinaryLinearQuantumFirstLAYER(in_features=16, out_features=16, bias=False)
  (fc1): BinaryLinearClassic(in_features=16, out_features=2, bias=False)
  (qca1): QC_Norm_Correction_try2()
)


In [5]:
if os.path.isfile(resume_path):
    print("=> loading checkpoint from '{}'<=".format(resume_path))
    checkpoint = torch.load(resume_path, map_location=device)
    epoch_init,acc = checkpoint["epoch"],checkpoint["acc"]
    model.load_state_dict(checkpoint["state_dict"])
    scheduler.load_state_dict(checkpoint["scheduler"])    
    scheduler.milestones = Counter(milestones)
    optimizer.load_state_dict(checkpoint["optimizer"])
else:
    epoch_init,acc = 0,0





if training:
    for epoch in range(epoch_init, max_epoch + 1):
        print("="*20,epoch,"epoch","="*20)  
        print("Epoch Start at:",time.strftime("%m/%d/%Y %H:%M:%S"))        

        print("-"*20,"learning rates","-"*20)
        for param_group in optimizer.param_groups:
            print(param_group['lr'],end=",")
        print()    
        
        print("-"*20,"training","-"*20)
        print("Trainign Start at:",time.strftime("%m/%d/%Y %H:%M:%S"))
        train(epoch)
        print("Trainign End at:",time.strftime("%m/%d/%Y %H:%M:%S"))
        print("-"*60)
        
        print()
        
        
        
        
        
        print("-"*20,"testing","-"*20)
        print("Testing Start at:",time.strftime("%m/%d/%Y %H:%M:%S"))        
        cur_acc = test()
        print("Testing End at:",time.strftime("%m/%d/%Y %H:%M:%S"))
        print("-"*60)
        print()
        
        scheduler.step()
        
        is_best = False
        if cur_acc > acc:
            is_best = True
            acc=cur_acc
        
        print("Best accuracy: {}; Current accuracy {}. Checkpointing".format(acc,cur_acc))
        save_checkpoint({
          'epoch': epoch + 1,
          'acc': acc, 
          'state_dict': model.state_dict(),      
          'optimizer' : optimizer.state_dict(),
           'scheduler': scheduler.state_dict(),
        }, is_best, save_path, 'checkpoint_{}_{}.pth.tar'.format(epoch,round(cur_acc,4)))
        print("Epoch End at:",time.strftime("%m/%d/%Y %H:%M:%S"))
        print("="*60)
        print()        
else:    
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, 
        num_workers=num_workers, shuffle=True, drop_last=True)
    test()
    

==================== 0 epoch ====================
Epoch Start at: 11/11/2020 18:29:56
-------------------- learning rates --------------------
0.01,
-------------------- training --------------------
Trainign Start at: 11/11/2020 18:29:56
Train Epoch: 0 [0/12049 (0%)]	Loss: 0.694402	Accuracy: 8/16 (50.00%)
Train Epoch: 0 [1600/12049 (13%)]	Loss: 0.668814	Accuracy: 803/1616 (49.69%)
Train Epoch: 0 [3200/12049 (27%)]	Loss: 0.678215	Accuracy: 1654/3216 (51.43%)
Train Epoch: 0 [4800/12049 (40%)]	Loss: 0.669456	Accuracy: 2429/4816 (50.44%)
Train Epoch: 0 [6400/12049 (53%)]	Loss: 0.678210	Accuracy: 3184/6416 (49.63%)
Train Epoch: 0 [8000/12049 (66%)]	Loss: 0.698141	Accuracy: 3956/8016 (49.35%)
Train Epoch: 0 [9600/12049 (80%)]	Loss: 0.673458	Accuracy: 4735/9616 (49.24%)
Train Epoch: 0 [11200/12049 (93%)]	Loss: 0.677545	Accuracy: 5517/11216 (49.19%)
-------------------- training done, loss --------------------
Training Set: Average loss: 0.674826
Trainign End at: 11/11/2020 18:30:02
---------

KeyboardInterrupt: 